In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
station_rain = pd.read_excel('../遥测站降雨数据.xlsx')
station_rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46056 entries, 0 to 46055
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   TimeStample  46056 non-null  datetime64[ns]
 1   R1           46056 non-null  float64       
 2   R2           46056 non-null  float64       
 3   R3           46056 non-null  float64       
 4   R4           46056 non-null  float64       
 5   R5           46056 non-null  float64       
 6   R6           46056 non-null  float64       
 7   R7           46056 non-null  float64       
 8   R8           46056 non-null  float64       
 9   R9           46056 non-null  float64       
 10  R10          46056 non-null  float64       
 11  R11          46056 non-null  float64       
 12  R12          46056 non-null  float64       
 13  R13          46056 non-null  float64       
 14  R14          46056 non-null  float64       
 15  R15          46056 non-null  float64       
 16  R16 

In [3]:
rain = pd.read_excel('../降雨预报数据.xlsx')
rain['dt'] = rain['TimeStample'].dt.strftime('%Y-%m-%d')
rain.drop('TimeStample', axis=1, inplace=True)
rain.head()

,D1,D2,D3,D4,D5,dt
0,0.0625,0.018182,0.014286,0.10,0.1,2013-03-11
1,0.1250,0.000000,0.014286,0.20,0.1,2013-03-12
2,0.0250,0.090909,0.142857,0.10,0.0,2013-03-13
3,0.0375,0.181818,0.071429,0.00,0.1,2013-03-14
4,0.1000,0.036364,0.000000,0.16,0.1,2013-03-15


In [4]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   D1      1836 non-null   float64
 1   D2      1836 non-null   float64
 2   D3      1836 non-null   float64
 3   D4      1836 non-null   float64
 4   D5      1836 non-null   float64
 5   dt      1836 non-null   object 
dtypes: float64(5), object(1)
memory usage: 86.2+ KB


In [5]:
environment = pd.read_excel('../环境表.xlsx')
environment.rename(columns={'TimeStample': 'dt'}, inplace=True)
environment.head()

,dt,T,w,wd
0,2013-01-01,0.168571,0.173913,999012
1,2013-01-02,0.157143,0.478261,999004
2,2013-01-03,0.128571,0.717391,999004
3,2013-01-04,0.037143,0.304348,999003
4,2013-01-05,0.071429,0.239130,999003


In [6]:
environment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919 entries, 0 to 1918
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   dt      1919 non-null   object 
 1   T       1914 non-null   float64
 2   w       1914 non-null   float64
 3   wd      1919 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 60.1+ KB


In [7]:
water = pd.read_excel('../入库流量数据.xlsx')
water['dt'] = water['TimeStample'].dt.strftime('%Y-%m-%d')
water.head()

,TimeStample,Qi,dt
0,2013-01-01 02:00:00,0.018201,2013-01-01
1,2013-01-01 05:00:00,0.018196,2013-01-01
2,2013-01-01 08:00:00,0.030095,2013-01-01
3,2013-01-01 11:00:00,0.123196,2013-01-01
4,2013-01-01 14:00:00,0.133178,2013-01-01


In [8]:
water.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15019 entries, 0 to 15018
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   TimeStample  15019 non-null  datetime64[ns]
 1   Qi           15019 non-null  float64       
 2   dt           15019 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 352.1+ KB


In [9]:
train1 = water.loc[(water['TimeStample'] >= '2017-01-01 02:00:00') & (water['TimeStample'] < '2017-02-01 00:00:00')]
train2 = water.loc[(water['TimeStample'] >= '2017-07-01 02:00:00') & (water['TimeStample'] < '2017-08-01 00:00:00')]
train3 = water.loc[(water['TimeStample'] >= '2017-10-01 02:00:00') & (water['TimeStample'] < '2017-11-01 00:00:00')]
print(train1.shape, train2.shape, train3.shape)


test1 = water.loc[(water['TimeStample'] >= '2017-02-01 02:00:00') & (water['TimeStample'] <= '2017-02-07 23:00:00')]
test2 = water.loc[(water['TimeStample'] >= '2017-08-01 02:00:00') & (water['TimeStample'] <= '2017-08-07 23:00:00')]
test3 = water.loc[(water['TimeStample'] >= '2017-11-01 02:00:00') & (water['TimeStample'] <= '2017-11-07 23:00:00')]
print(test1.shape, test2.shape, test3.shape)

(248, 3) (248, 3) (248, 3)
(56, 3) (56, 3) (56, 3)


In [10]:
df_list = [train1, train2, train3, test1, test2, test3]
data = pd.concat(df_list)
data.shape

(912, 3)

In [11]:
# train1 = df.loc[(df['TimeStample'] >= '2018-01-01 02:00:00') & (df['TimeStample'] < '2018-02-01 00:00:00')]
# train2 = df.loc[(df['TimeStample'] >= '2018-07-01 02:00:00') & (df['TimeStample'] < '2018-08-01 00:00:00')]
# train3 = df.loc[(df['TimeStample'] >= '2018-10-01 02:00:00') & (df['TimeStample'] < '2018-11-01 00:00:00')]
# train1.shape, train2.shape, train3.shape


# test1 = df.loc[(df['TimeStample'] >= '2018-02-01 02:00:00') & (df['TimeStample'] <= '2018-02-07 23:00:00')]
# test2 = df.loc[(df['TimeStample'] >= '2018-08-01 02:00:00') & (df['TimeStample'] <= '2018-08-07 23:00:00')]
# test3 = df.loc[(df['TimeStample'] >= '2018-11-01 02:00:00') & (df['TimeStample'] <= '2018-11-07 23:00:00')]
# test1.shape, test2.shape, test3.shape

In [12]:
data = data.merge(rain, how='left', on='dt')
data = data.merge(environment, how='left', on='dt')
data = data.merge(station_rain, how='left', on='TimeStample')
data.head()

,TimeStample,Qi,dt,D1,D2,D3,D4,D5,T,w,...,R30,R31,R32,R33,R34,R35,R36,R37,R38,R39
0,2017-01-01 02:00:00,0.083045,2017-01-01,0.025,0.036364,0.114286,0.24,0.24,0.342857,0.123913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01 05:00:00,0.087865,2017-01-01,0.025,0.036364,0.114286,0.24,0.24,0.342857,0.123913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01 08:00:00,0.074158,2017-01-01,0.025,0.036364,0.114286,0.24,0.24,0.342857,0.123913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01 11:00:00,0.062911,2017-01-01,0.025,0.036364,0.114286,0.24,0.24,0.342857,0.123913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01 14:00:00,0.040116,2017-01-01,0.025,0.036364,0.114286,0.24,0.24,0.342857,0.123913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
train1 = data.loc[(data['TimeStample'] >= '2017-01-01 02:00:00') & (data['TimeStample'] < '2017-02-01 00:00:00')]
train2 = data.loc[(data['TimeStample'] >= '2017-07-01 02:00:00') & (data['TimeStample'] < '2017-08-01 00:00:00')]
train3 = data.loc[(data['TimeStample'] >= '2017-10-01 02:00:00') & (data['TimeStample'] < '2017-11-01 00:00:00')]
print(train1.shape, train2.shape, train3.shape)


test1 = data.loc[(data['TimeStample'] >= '2017-02-01 02:00:00') & (data['TimeStample'] <= '2017-02-07 23:00:00')]
test2 = data.loc[(data['TimeStample'] >= '2017-08-01 02:00:00') & (data['TimeStample'] <= '2017-08-07 23:00:00')]
test3 = data.loc[(data['TimeStample'] >= '2017-11-01 02:00:00') & (data['TimeStample'] <= '2017-11-07 23:00:00')]
print(test1.shape, test2.shape, test3.shape)

(248, 50) (248, 50) (248, 50)
(56, 50) (56, 50) (56, 50)


In [14]:
train_y1 = train1['Qi']
train1.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
train_x1 = train1

train_y2 = train2['Qi']
train2.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
train_x2 = train2

train_y3 = train3['Qi']
train3.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
train_x3 = train3

test_y1 = test1['Qi']
test1.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
test_x1 = test1

test_y2 = test2['Qi']
test2.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
test_x2 = test2

test_y3 = test3['Qi']
test3.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
test_x3 = test3

In [15]:
lgb_train1 = lgb.Dataset(train_x1, label=train_y1)
lgb_test1 = lgb.Dataset(test_x1, label=test_y1, reference=lgb_train1)

lgb_train2 = lgb.Dataset(train_x2, label=train_y2)
lgb_test2 = lgb.Dataset(test_x2, label=test_y2, reference=lgb_train2)

lgb_train3 = lgb.Dataset(train_x3, label=train_y3)
lgb_test3 = lgb.Dataset(test_x3, label=test_y3, reference=lgb_train1)

In [16]:
# regression
params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'mae',
          'learning_rate': 0.05}

In [17]:
lgb_model1 = lgb.train(params,
                       lgb_train1,
                       valid_sets=[lgb_test1, lgb_train1],
                       num_boost_round=1000,
                       early_stopping_rounds=200,
                       verbose_eval=10)

Training until validation scores don't improve for 200 rounds
[10]	training's l1: 0.0263967	valid_0's l1: 0.0352662
[20]	training's l1: 0.0258617	valid_0's l1: 0.0349808
[30]	training's l1: 0.0255003	valid_0's l1: 0.0345366
[40]	training's l1: 0.0252925	valid_0's l1: 0.0344783
[50]	training's l1: 0.0251229	valid_0's l1: 0.0344158
[60]	training's l1: 0.0250247	valid_0's l1: 0.0344374
[70]	training's l1: 0.0249597	valid_0's l1: 0.0343972
[80]	training's l1: 0.0249167	valid_0's l1: 0.0344002
[90]	training's l1: 0.024883	valid_0's l1: 0.0344191
[100]	training's l1: 0.0248586	valid_0's l1: 0.0344538
[110]	training's l1: 0.0248468	valid_0's l1: 0.0344649
[120]	training's l1: 0.0248419	valid_0's l1: 0.03449
[130]	training's l1: 0.0248355	valid_0's l1: 0.0345165
[140]	training's l1: 0.0248313	valid_0's l1: 0.0345221
[150]	training's l1: 0.0248278	valid_0's l1: 0.0345196
[160]	training's l1: 0.0248236	valid_0's l1: 0.0345282
[170]	training's l1: 0.0248213	valid_0's l1: 0.034519
[180]	training's

In [18]:
lgb_model2 = lgb.train(params,
                       lgb_train2,
                       valid_sets=[lgb_test2, lgb_train2],
                       num_boost_round=1000,
                       early_stopping_rounds=200,
                       verbose_eval=10)

Training until validation scores don't improve for 200 rounds
[10]	training's l1: 0.102614	valid_0's l1: 0.112615
[20]	training's l1: 0.0892049	valid_0's l1: 0.103721
[30]	training's l1: 0.0810191	valid_0's l1: 0.100914
[40]	training's l1: 0.0761696	valid_0's l1: 0.099476
[50]	training's l1: 0.0730821	valid_0's l1: 0.0975911
[60]	training's l1: 0.0711617	valid_0's l1: 0.0996174
[70]	training's l1: 0.0696579	valid_0's l1: 0.10195
[80]	training's l1: 0.0686154	valid_0's l1: 0.101774
[90]	training's l1: 0.0674399	valid_0's l1: 0.10069
[100]	training's l1: 0.0664245	valid_0's l1: 0.100629
[110]	training's l1: 0.0655692	valid_0's l1: 0.100652
[120]	training's l1: 0.0648445	valid_0's l1: 0.100055
[130]	training's l1: 0.0642333	valid_0's l1: 0.100125
[140]	training's l1: 0.0637409	valid_0's l1: 0.0998254
[150]	training's l1: 0.0631031	valid_0's l1: 0.100699
[160]	training's l1: 0.0624369	valid_0's l1: 0.100747
[170]	training's l1: 0.0618462	valid_0's l1: 0.100825
[180]	training's l1: 0.061425

In [19]:
lgb_model3 = lgb.train(params,
                       lgb_train3,
                       valid_sets=[lgb_test3, lgb_train3],
                       num_boost_round=1000,
                       early_stopping_rounds=200,
                       verbose_eval=10)

Training until validation scores don't improve for 200 rounds
[10]	training's l1: 0.0791693	valid_0's l1: 0.207431
[20]	training's l1: 0.0642511	valid_0's l1: 0.160702
[30]	training's l1: 0.0574338	valid_0's l1: 0.132723
[40]	training's l1: 0.0540993	valid_0's l1: 0.118753
[50]	training's l1: 0.0520311	valid_0's l1: 0.108821
[60]	training's l1: 0.0501259	valid_0's l1: 0.102212
[70]	training's l1: 0.0493899	valid_0's l1: 0.0956335
[80]	training's l1: 0.0486963	valid_0's l1: 0.0909112
[90]	training's l1: 0.0481685	valid_0's l1: 0.0869541
[100]	training's l1: 0.0476372	valid_0's l1: 0.0834952
[110]	training's l1: 0.047361	valid_0's l1: 0.082489
[120]	training's l1: 0.0469642	valid_0's l1: 0.0818064
[130]	training's l1: 0.0466401	valid_0's l1: 0.082168
[140]	training's l1: 0.0464256	valid_0's l1: 0.0809156
[150]	training's l1: 0.0459885	valid_0's l1: 0.080837
[160]	training's l1: 0.0457882	valid_0's l1: 0.0804571
[170]	training's l1: 0.0455958	valid_0's l1: 0.0801862
[180]	training's l1: 0

In [20]:
train1 = water.loc[(water['TimeStample'] >= '2018-01-01 02:00:00') & (water['TimeStample'] < '2018-02-01 00:00:00')]
train2 = water.loc[(water['TimeStample'] >= '2018-07-01 02:00:00') & (water['TimeStample'] < '2018-08-01 00:00:00')]
train3 = water.loc[(water['TimeStample'] >= '2018-10-01 02:00:00') & (water['TimeStample'] < '2018-11-01 00:00:00')]
print(train1.shape, train2.shape, train3.shape)


test1 = pd.DataFrame({'TimeStample': pd.date_range(start='2018-02-01 02:00:00', periods=56, freq='3h')})
test2 = pd.DataFrame({'TimeStample': pd.date_range(start='2018-08-01 02:00:00', periods=56, freq='3h')})
test3 = pd.DataFrame({'TimeStample': pd.date_range(start='2018-11-01 02:00:00', periods=56, freq='3h')})
print(test1.shape, test2.shape, test3.shape)

(248, 3) (248, 3) (248, 3)
(56, 1) (56, 1) (56, 1)


In [21]:
df_list = [train1, train2, train3, test1, test2, test3]
data = pd.concat(df_list)
data.shape

(912, 3)

In [22]:
data = data.merge(rain, how='left', on='dt')
data = data.merge(environment, how='left', on='dt')
data = data.merge(station_rain, how='left', on='TimeStample')
data.head()

,TimeStample,Qi,dt,D1,D2,D3,D4,D5,T,w,...,R30,R31,R32,R33,R34,R35,R36,R37,R38,R39
0,2018-01-01 02:00:00,0.020184,2018-01-01,0.0625,0.272727,0.214286,0.1,0.1,0.308571,0.065217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01 05:00:00,0.025757,2018-01-01,0.0625,0.272727,0.214286,0.1,0.1,0.308571,0.065217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-01 08:00:00,0.022995,2018-01-01,0.0625,0.272727,0.214286,0.1,0.1,0.308571,0.065217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-01 11:00:00,0.015113,2018-01-01,0.0625,0.272727,0.214286,0.1,0.1,0.308571,0.065217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-01 14:00:00,0.017975,2018-01-01,0.0625,0.272727,0.214286,0.1,0.1,0.308571,0.065217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train1 = data.loc[(data['TimeStample'] >= '2018-01-01 02:00:00') & (data['TimeStample'] < '2018-02-01 00:00:00')]
train2 = data.loc[(data['TimeStample'] >= '2018-07-01 02:00:00') & (data['TimeStample'] < '2018-08-01 00:00:00')]
train3 = data.loc[(data['TimeStample'] >= '2018-10-01 02:00:00') & (data['TimeStample'] < '2018-11-01 00:00:00')]
print(train1.shape, train2.shape, train3.shape)

test1 = data.loc[(data['TimeStample'] >= '2018-02-01 02:00:00') & (data['TimeStample'] <= '2018-02-07 23:00:00')]
test2 = data.loc[(data['TimeStample'] >= '2018-08-01 02:00:00') & (data['TimeStample'] <= '2018-08-07 23:00:00')]
test3 = data.loc[(data['TimeStample'] >= '2018-11-01 02:00:00') & (data['TimeStample'] <= '2018-11-07 23:00:00')]
print(test1.shape, test2.shape, test3.shape)

(248, 50) (248, 50) (248, 50)
(56, 50) (56, 50) (56, 50)


In [24]:
train_y1 = train1['Qi']
train1.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
train_x1 = train1

train_y2 = train2['Qi']
train2.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
train_x2 = train2

train_y3 = train3['Qi']
train3.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
train_x3 = train3

# test_y1 = test1['Qi']
sub1 = test1[['TimeStample']]
test1.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
test_x1 = test1

# test_y2 = test2['Qi']
sub2 = test2[['TimeStample']]
test2.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
test_x2 = test2

# test_y3 = test3['Qi']
sub3 = test3[['TimeStample']]
test3.drop(['TimeStample', 'dt', 'Qi'], axis=1, inplace=True)
test_x3 = test3

In [25]:
lgb_train1 = lgb.Dataset(train_x1, label=train_y1)
# lgb_test1 = lgb.Dataset(test_x1, label=test_y1, reference=lgb_train1)

lgb_train2 = lgb.Dataset(train_x2, label=train_y2)
# lgb_test2 = lgb.Dataset(test_x2, label=test_y2, reference=lgb_train2)

lgb_train3 = lgb.Dataset(train_x3, label=train_y3)
# lgb_test3 = lgb.Dataset(test_x3, label=test_y3, reference=lgb_train1)

In [28]:
lgb_model1 = lgb.train(params,
                       lgb_train1,
#                        valid_sets=[lgb_test1, lgb_train1],
                       num_boost_round=lgb_model1.best_iteration,
                       verbose_eval=10)

pred1 = lgb_model1.predict(test_x1)
sub1 = pd.DataFrame(pred1).T
sub1.columns = ['Prediction{}'.format(i) for i in range(1, 57)]
sub1['SeqNum'] = 1
l = ['SeqNum'] + ['Prediction{}'.format(i) for i in range(1, 57)]
sub1 = sub1[l]

In [29]:
lgb_model2 = lgb.train(params,
                       lgb_train2,
#                        valid_sets=[lgb_test1, lgb_train1],
                       num_boost_round=lgb_model2.best_iteration,
                       verbose_eval=10)

pred2 = lgb_model2.predict(test_x2)
sub2 = pd.DataFrame(pred2).T
sub2.columns = ['Prediction{}'.format(i) for i in range(1, 57)]
sub2['SeqNum'] = 2
l = ['SeqNum'] + ['Prediction{}'.format(i) for i in range(1, 57)]
sub2 = sub2[l]

In [30]:
lgb_model3 = lgb.train(params,
                       lgb_train3,
#                        valid_sets=[lgb_test3, lgb_train3],
                       num_boost_round=lgb_model3.best_iteration,
                       verbose_eval=10)

pred3 = lgb_model1.predict(test_x3)
sub3 = pd.DataFrame(pred3).T
sub3.columns = ['Prediction{}'.format(i) for i in range(1, 57)]
sub3['SeqNum'] = 3
l = ['SeqNum'] + ['Prediction{}'.format(i) for i in range(1, 57)]
sub3 = sub3[l]

In [31]:
sub = pd.concat([sub1, sub2, sub3])
sub

,SeqNum,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7,Prediction8,Prediction9,...,Prediction47,Prediction48,Prediction49,Prediction50,Prediction51,Prediction52,Prediction53,Prediction54,Prediction55,Prediction56
0,1,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,...,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199
0,2,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,...,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657,0.476657
0,3,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,...,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199,0.102199


In [32]:
sub.to_csv('sub.csv', index=False)